In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.linear_model import LogisticRegression

import matplotlib.pyplot as plt
import seaborn as sns

from keras.layers import Dense, Input, Lambda, BatchNormalization, Activation
from keras.losses import mse, binary_crossentropy, kullback_leibler_divergence
from keras.models import Model
from keras.utils import plot_model
from keras import optimizers
from keras.callbacks import EarlyStopping

from IPython.display import SVG

from keras import backend as K

import os
os.environ["KMP_DUPLICATE_LIB_OK"]="TRUE"

Using TensorFlow backend.


## Import datasets

In [2]:
# Train data
X_brca_train = pd.read_csv("data/ciriello_brca_filtered_train.csv")
X_brca_train = X_brca_train[X_brca_train.Ciriello_subtype != "Normal"]

y_brca_train = X_brca_train["Ciriello_subtype"]

X_brca_train.drop(['Ciriello_subtype'], axis="columns", inplace=True)

X_tcga_no_brca = pd.read_csv("data/tcga_filtered_no_brca.csv")

# Test data
X_brca_test = pd.read_csv("data/tcga_brca_filtered_test.csv")
X_brca_test = X_brca_test[X_brca_test.subtype != "Normal"]
y_brca_test = X_brca_test["subtype"]

X_brca_test.drop(['subtype'], axis="columns", inplace=True)

## Define Variational Autoencoder auxiliary functions

In [3]:
def sampling(args):
    
    """Reparameterization trick by sampling from an isotropic unit Gaussian.
    # Arguments
        args (tensor): mean and log of variance of Q(z|X)
    # Returns
        z (tensor): sampled latent vector
    """
    
    z_mean, z_log_var = args
    batch = K.shape(z_mean)[0]
    dim = K.int_shape(z_mean)[1]
    # by default, random_normal has mean = 0 and std = 1.0
    epsilon = K.random_normal(shape=(batch, dim), mean=0., stddev=1.)
    return z_mean + K.exp(0.5 * z_log_var) * epsilon

def vae_loss(y_true, y_pred):
    # E[log P(X|z)]
    reconstruction_loss = original_dim * binary_crossentropy(y_true, y_pred) # because it returns the mean cross-entropy
    # reconstruction_loss = mse(y_true, y_pred)
    # D_KL(Q(z|X) || P(z|X)); calculate in closed form as both dist. are Gaussian
    kl_loss = -0.5 * K.sum(1. + z_log_var_encoded - K.exp(z_log_var_encoded) - K.square(z_mean_encoded), axis=1)

    return K.mean(reconstruction_loss + kl_loss)

## Model implementation

In [4]:
def train_model(X_autoencoder, X_train_logreg, y_train_logreg, X_val_logreg, y_val_logreg, intermediate_dim=100, latent_dim=100):
    
    #Initialize variables and hyperparameters
    input_shape = (original_dim,)
    intermediate_dim = intermediate_dim
    latent_dim = latent_dim

    batch_size = 100
    epochs = 150 
    learning_rate = 0.001

    #Implement Variational Autoencoder
    
    # Build Encoder
    inputs = Input(shape=input_shape, name='encoder_input')
    hidden_dense = Dense(intermediate_dim)(inputs)
    hidden_dense_batchnorm = BatchNormalization()(hidden_dense)
    hidden_dense_encoded = Activation('relu')(hidden_dense_batchnorm)

    z_mean_dense = Dense(latent_dim, name='z_mean')(hidden_dense_encoded)
    z_log_var_dense = Dense(latent_dim, name='z_log_var')(hidden_dense_encoded)
    
    z_mean_dense_batchnorm = BatchNormalization()(z_mean_dense)
    z_mean_encoded = Activation('relu')(z_mean_dense_batchnorm)
    
    z_log_var_dense_batchnorm = BatchNormalization()(z_log_var_dense)
    z_log_var_encoded = Activation('relu')(z_log_var_dense_batchnorm)

    z = Lambda(sampling, output_shape=(latent_dim,), name='z')([z_mean_encoded, z_log_var_encoded])

    encoder = Model(inputs, [z_mean_encoded, z_log_var_encoded, z], name='encoder')
    
    # Build Decoder
    latent_inputs = Input(shape=(latent_dim,), name='z_sampling')
    decoder_hidden = Dense(intermediate_dim, activation='relu', name='decoder_hidden')(latent_inputs)
    outputs = Dense(original_dim, activation='sigmoid', name='decoder_output')(decoder_hidden)

    decoder = Model(latent_inputs, outputs, name='decoder')
    
    #Build Variational Autoencoder
    outputs = decoder(encoder(inputs)[2]) # fetches the z layer, the sampled one
    vae = Model(inputs, outputs, name='vae')

    adam = optimizers.Adam(lr=learning_rate)
    
    def vae_loss(y_true, y_pred):
        # E[log P(X|z)]
        reconstruction_loss = original_dim * binary_crossentropy(y_true, y_pred) # because it returns the mean cross-entropy
        # reconstruction_loss = mse(y_true, y_pred)
        # D_KL(Q(z|X) || P(z|X)); calculate in closed form as both dist. are Gaussian
        kl_loss = -0.5 * K.sum(1 + z_log_var_encoded - K.exp(z_log_var_encoded) - K.square(z_mean_encoded), axis=1)

        return K.mean(reconstruction_loss + kl_loss)
    
    vae.compile(optimizer=adam, loss=vae_loss)
    
    # Split validation set
    test_set_percent = 0.1
    X_autoencoder_val = X_autoencoder.sample(frac=test_set_percent)
    X_autoencoder_train = X_autoencoder.drop(X_autoencoder_val.index)
    
    # Limit number of CPU cores used
    K.set_session(K.tf.Session(config=K.tf.ConfigProto(intra_op_parallelism_threads=10, inter_op_parallelism_threads=10)))
    
    # Train the model
    
    fit_hist = vae.fit(X_autoencoder_train, 
                    X_autoencoder_train,
                    shuffle=True,
                    epochs=epochs,
                    verbose=2,
                    batch_size=batch_size,
                    callbacks=[EarlyStopping(monitor='val_loss', patience=10)],
                    validation_data=(X_autoencoder_val, X_autoencoder_val))
    
    encoded_train_logreg = encoder.predict(X_train_logreg)
    encoded_train_logreg = pd.DataFrame(encoded_train_logreg[0])

    encoded_val_logreg = encoder.predict(X_val_logreg)
    encoded_val_logreg = pd.DataFrame(encoded_val_logreg[0])
    
    clf = LogisticRegression(random_state=0, solver='liblinear', penalty="l1", C=1, multi_class="auto").fit(encoded_train_logreg, y_train_logreg)
    
    history_df = pd.DataFrame(fit_hist.history)
    score = clf.score(encoded_val_logreg, y_val_logreg)
    conf_matrix = confusion_matrix(y_val_logreg, clf.predict(encoded_val_logreg))
    
    return history_df, score, conf_matrix

## Cross Validation for the whole model - 5-Fold Cross-Validation

In [ ]:
history = {}
scores = []
confusion_matrixes = []


skf = StratifiedKFold(n_splits=5)
i=1

for train_index, test_index in skf.split(X_brca_train, y_brca_train):
    print('Fold {} of {}'.format(i, skf.n_splits))
    
    X_train, X_val = X_brca_train.iloc[train_index], X_brca_train.iloc[test_index]
    y_train, y_val = y_brca_train.iloc[train_index], y_brca_train.iloc[test_index]
    
    # Prepare data to train Variational Autoencoder (merge dataframes and normalize)
    X_autoencoder = pd.concat([X_train, X_tcga_no_brca], sort=True)
    scaler = MinMaxScaler()
    scaler.fit(X_autoencoder)
    X_autoencoder_scaled = pd.DataFrame(scaler.transform(X_autoencoder), columns=X_autoencoder.columns)

    # Scale logistic regression data
    scaler.fit(X_brca_train)
    X_brca_train_scaled = pd.DataFrame(scaler.transform(X_brca_train), columns=X_brca_train.columns)
    X_brca_test_scaled = pd.DataFrame(scaler.transform(X_brca_test), columns=X_brca_test.columns)
    
    
    # Because it is used by the error function
    original_dim = X_autoencoder.shape[1]
    
    #Train the Model
    hist, score, conf_matrix = train_model(X_autoencoder=X_autoencoder_scaled, 
                                           X_train_logreg=X_train, 
                                           y_train_logreg=y_train, 
                                           X_val_logreg=X_val, 
                                           y_val_logreg=y_val,
                                           intermediate_dim=100,
                                           latent_dim=100)
    
    print(score)
    history[i-1] = hist
    scores.append(score)
    confusion_matrixes.append(conf_matrix)
    i+=1
print('5-Fold results: {}'.format(scores))
print('Latent dim: {}, Accuracy: {}'.format(latent_dim, np.mean(scores)))
    

Fold 1 of 5
Train on 9908 samples, validate on 1101 samples
Epoch 1/150
 - 31s - loss: 10379.9346 - val_loss: 10194.8759
Epoch 2/150
 - 28s - loss: 10038.9273 - val_loss: 10070.5314
Epoch 3/150
 - 27s - loss: 9987.0178 - val_loss: 10009.2176
Epoch 4/150
 - 28s - loss: 9957.1261 - val_loss: 9996.7418
Epoch 5/150
 - 27s - loss: 9938.0894 - val_loss: 9980.0530
Epoch 6/150
 - 28s - loss: 9925.5694 - val_loss: 9965.4989
Epoch 7/150
 - 28s - loss: 9916.8586 - val_loss: 9956.3913
Epoch 8/150
